### Creating Databases and Tables

### Creating Databases
- Varies by the database type
- Databases like PostgreSQL and MySQL have command line tools to initialize the database
- With SQLite, the `create_engine()` statement will create the database and file if they do not already exist

### Building a Table

In [1]:
from sqlalchemy import (Table, Column, String, Integer, Float, Boolean)
from sqlalchemy import create_engine, MetaData
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

In [2]:
# employees = Table('employees', metadata,
#                  Column('id', Integer()),
#                  Column('name', String(225)),
#                  Column('salary', DECIMAL()),
#                  Column('active', Boolean()))

# metadata.create_all(engine)
# engine.table_names()

### Creating Tables
- Still uses the Table object like we did for reflection
- Replaces the autoload keyword arguments with Column objects
- Creates the tables in the actual database by using the `create_all()` method on the MetaData instance
- Other tools are needed to handle database table updates, such as Alembic or raw SQL

### Creating Tables - Additional Column Options
- `unique` forces all values for the data in a column to be unique
- `nullable` determines if a column can be empty in a row
- `default` sets a default value if one isn't supplied

In [3]:
employees = Table('employees', metadata,
                 Column('id', Integer()),
                 Column('name', String(225), unique=True,
                       nullable=False),
                 Column('salary', Float(), default=100.00),
                 Column('active', Boolean(), default=True))

employees.constraints

{CheckConstraint(<sqlalchemy.sql.elements.BinaryExpression object at 0x000000000581D128>, name='_unnamed_', table=Table('employees', MetaData(bind=None), Column('id', Integer(), table=<employees>), Column('name', String(length=225), table=<employees>, nullable=False), Column('salary', Float(), table=<employees>, default=ColumnDefault(100.0)), Column('active', Boolean(), table=<employees>, default=ColumnDefault(True)), schema=None), _create_rule=<sqlalchemy.util.langhelpers.portable_instancemethod object at 0x000000000581B3F0>, _type_bound=True),
 PrimaryKeyConstraint(),
 UniqueConstraint(Column('name', String(length=225), table=<employees>, nullable=False))}

In [4]:
metadata.create_all(engine)
engine.table_names()

['employees']

### Inserting Data into a Table

### Adding Data to a Table
- Done with the `insert()` statement
- `Insert()` takes the table we are loading data into as the argument
- We add all the values we want to insert in with the `values` clause as `column=value` pairs
- Doesn't return any rows, so no need for a fetch method.

### Inserting One Row

In [5]:
connection = engine.connect()

In [6]:
from sqlalchemy import insert

stmt = insert(employees).values(id=1,
                               name='Jason',
                               salary=1.00,
                               active=True)

result_proxy = connection.execute(stmt)

In [7]:
print(result_proxy.rowcount)

1


### Inserting Multiple Rows
- Build an insert statement without any values
- Build a list of dictionaries that represent all the values clauses for the rows you want to insert
- Pass both the stmt and the values list to the execute method on connection

In [8]:
stmt = insert(employees)

values_list = [ 
            {'id': 2, 'name': 'Rebecca', 'salary': 2.00, 'active': True},
            {'id': 3, 'name': 'Bob', 'salary': 0.00, 'active': False}]

result_proxy = connection.execute(stmt, values_list)

print(result_proxy.rowcount)

2


In [9]:
from sqlalchemy import select
stmt = select([employees])
print(connection.execute(stmt).fetchall())

[(1, 'Jason', 1.0, True), (2, 'Rebecca', 2.0, True), (3, 'Bob', 0.0, False)]


In [10]:
census = Table('census', metadata,
            Column('state', String(50)), 
            Column('sex', String(10)), 
            Column('age', String(10)),
            Column('pop2000', String(50)),
            Column('pop2008', String(50)))

metadata.create_all(engine)
engine.table_names()

['census', 'employees']

In [11]:
import csv 

In [12]:
# Create a insert statement for census: stmt
stmt = insert(census)

# Create an empty list and zeroed row count: values_list, total_rowcount
values_list = []
total_rowcount = 0

# Enumerate the rows of csv_reader
with open('census.csv', newline='\n') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    for idx, row in enumerate(csv_reader):
        data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3], 'pop2008': row[4]}
        values_list.append(data)

        if idx % 51 == 0:
            results = connection.execute(stmt, values_list)
            total_rowcount += results.rowcount
            values_list = []
print(total_rowcount)


8722


In [14]:
stmt = select([census])
print(connection.execute(stmt).first())

('Illinois', 'M', '0', '89600', '95012')
